#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [ ]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df


## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten.

### Expected Result:


| d         | match       |
------------|--------------
| :Ceres    | dbr:1_Ceres |
| :Eris     |             |
| :Haumea   |             |
| :Makemake |             |
| :Pluto    | dbr:Pluto   |


In [ ]:
df = sparql_select(g,"""
base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT *
WHERE { 
   ?d rdf:type dbo:DwarfPlanet .
   OPTIONAL
   { 
      ?d skos:exactMatch ?match. 
   }
}
ORDER BY ?d ASC(?d)

""",use_prefixes=True)
df

## Task 2 (1 pt)
Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne.

### Expected Result:

| star             | match       | massVal   | massUoM     |
|------------------|-------------|-----------|-------------|
| :AlphaCentauriA  | wd:Q2090157 | 1.1       | v:SolarMass |
| :AlphaCentauriB  | wd:Q1052548 | 0.9       | v:SolarMass |
| :ProximaCentauri | wd:Q14266   | 0.1221    | v:SolarMass |
| :Sun             | dbr:Sun     | 1.9884E30 | unit:KG     |



In [ ]:
df = sparql_select(g,"""
base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?star ?match ?massVal ?massUoM
WHERE { 
   ?star  rdf:type dbo:Star .
   ?star skos:exactMatch ?match . 
   ?star v:mass [rdf:value ?massVal] .
   ?star v:mass [v:uom ?massUoM] .
}
ORDER BY ?star ASC(?star)

""",use_prefixes=True)
df

## Task 3 (1 pt)

Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten.

### Expected Result:

| planet   | assertedNo | noOfDescribedMoons |
|--|--|--|
| :Earth   | 1          | 1                  |
| :Jupiter | 79         | 4                  |
| :Mars    | 2          | 2                  |
| :Mercury | 0          | 0                  |
| :Neptune | 14         | 0                  |
| :Saturn  | 82         | 2                  |
| :Uranus  | 27         | 0                  |
| :Venus   | 0          | 0                  |


In [ ]:
df = sparql_select(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?planet ?assertedNo ?noOfDescribedMoons
WHERE { 
   ?planet rdf:type dbo:Planet .
   ?planet v:nrOfMoons ?assertedNo. 
   {
      SELECT ?planet (count(?moon) AS ?noOfDescribedMoons)
      WHERE
      { 
         ?planet rdf:type dbo:Planet .
         OPTIONAL
         { 
            ?moon rdf:type dbo:Satellite .
            ?moon v:orbits ?planet .
         }
      }
      GROUP BY ?planet
   }
}
ORDER BY ?planet ASC(?planet)

""",use_prefixes=True)
df

## Task 4 (1 pt)

Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet.

### Expected Result:

| avgNoOfDescribedMoons |
|-|
| 1.125                 |


In [ ]:
df = sparql_select(g,"""
base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT (avg(?noOfDescribedMoons) AS ?avgNoOfDescribedMoons)
WHERE { 
   ?planet rdf:type dbo:Planet .
   {
      SELECT ?planet (count(?moon) AS ?noOfDescribedMoons)
      WHERE
      { 
         ?planet rdf:type dbo:Planet .
         OPTIONAL
         { 
            ?moon rdf:type dbo:Satellite .
            ?moon v:orbits ?planet .
         }
      }
      GROUP BY ?planet
   }
}
ORDER BY ?planet ASC(?planet)

""",use_prefixes=False)
df

## Task 5 (1 pt)

Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen.

### Expected Result:

| class           | noOfInstances |
|--|--|
| dbo:DwarfPlanet | 5             |
| dbo:Planet      | 8             |
| dbo:Satellite   | 9             |
| dbo:Star        | 4             |


In [ ]:
df = sparql_select(g, """

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?class ?noOfInstances
WHERE
{
   {
      SELECT ?class (count(?class) AS ?noOfInstances)
      WHERE { 
      ?instance rdf:type ?class 
      }
      GROUP BY ?class
   }
   FILTER(?noOfInstances > 2)
}
ORDER BY ?class

""")

df

df2 = sparql_select(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?class (count(?class) AS ?noOfInstances)
WHERE { 
   ?instance rdf:type ?class .
}
GROUP BY ?class
HAVING(?noOfInstances > 2)
ORDER BY ?class ASC(?class)

""",use_prefixes=True)
df

## Task 6 (1 pt)

Geben Sie die im RDF-Graph verwendeten Properties und ihre Häufigkeit je Klasse aus. Gezählt werden sollen die Instanzen der Klasse, die die Property ausprägen. Geben Sie nur Properties zu Klassen aus, die von mindestens 7 Instanzen dieser Klasse ausgeprägt werden. Ordnen Sie die Ausgabe zuerst nach der Klasse und dann nach der Property.

### Expected Result:

| class         | prop            | noOfInstances |
|--|--|--|
| dbo:Planet    | v:apoapsis      | 8             |
| dbo:Planet    | v:nrOfMoons     | 8             |
| dbo:Planet    | v:orbits        | 8             |
| dbo:Planet    | v:radius        | 7             |
| dbo:Planet    | v:temperature   | 7             |
| dbo:Planet    | rdf:type        | 8             |
| dbo:Planet    | skos:exactMatch | 8             |
| dbo:Satellite | v:orbits        | 9             |
| dbo:Satellite | v:radius        | 9             |
| dbo:Satellite | rdf:type        | 9             |


In [ ]:
df = sparql_select(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?class ?prop ?noOfInstances
WHERE
{
   {
      SELECT ?class ?prop (count(DISTINCT ?instance) AS ?noOfInstances )
      WHERE { 
         ?instance rdf:type ?class .
         ?instance ?prop ?x
      }
      GROUP BY ?prop ?class
      ORDER BY ?class ?prop 
   }
   FILTER(?noOfInstances > 6)
}

""",use_prefixes=True)
df

df2 = sparql_select(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?class ?prop (count(DISTINCT ?instance) AS ?noOfInstances )
WHERE { 
   ?instance rdf:type ?class .
   ?instance ?prop ?x
}

GROUP BY ?prop ?class
HAVING(?noOfInstances > 6)
ORDER BY ?class ?prop 

""",use_prefixes=True)
df

## Task 7 (1 pt)

Ermitteln Sie alle Sterne, sowie alle Monde. Zu jedem Mond ermitteln Sie auch den Planet, den dieser Mond umrundet. Ordnen Sie die Ausgabe nach der URI der Sterne und Monde.

### Expected Result:

| x                | y        |
|--|--|
| :AlphaCentauriA  |          |
| :AlphaCentauriB  |          |
| :Callisto        | :Jupiter |
| :Deimos          | :Mars    |
| :Enceladus       | :Saturn  |
| :Europa          | :Jupiter |
| :Ganymede        | :Jupiter |
| :Io              | :Jupiter |
| :Mimas           | :Saturn  |
| :Moon            | :Earth   |
| :Phobos          | :Mars    |
| :ProximaCentauri |          |
| :Sun             |          |



In [ ]:
df = sparql_select(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

SELECT ?x ?y
WHERE 
{
   {?x rdf:type dbo:Star}
   UNION
   {?x rdf:type dbo:Satellite} 
   OPTIONAL
   {
      ?x v:orbits ?y
   }
}
ORDER BY ?x

""",use_prefixes=True)
df

## Task 8 (1 pt)

Erzeugen sie mittels einer Construct-Query den angezeigten RDF-GRAPH (Expected Result). Reihenfolge ist unerheblich.

### Expected Result:

```@prefix :     <http://dke.jku.at/example/solarsystem/> .
@prefix dbd:  <http://dbpedia.org/datatype/> .
@prefix dbo:  <http://dbpedia.org/ontology/> .
@prefix dbr:  <http://dbpedia.org/resource/> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sdo:  <http://schema.org/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix v:    <http://dke.jku.at/example/examplevocabulary/> .
@prefix wd:   <http://www.wikidata.org/entity/> .
@prefix wdp:  <http://www.wikidata.org/prop/> .

:Mimas  rdf:type  v:Himmelskoerper .

:Venus  rdf:type  v:Himmelskoerper .

:Phobos  rdf:type  v:Himmelskoerper .

:Deimos  rdf:type  v:Himmelskoerper .

:Jupiter  rdf:type         v:Himmelskoerper ;
        v:wirdUmrundetVon  :Europa , :Ganymede , :Io , :Callisto .

:Mars   rdf:type           v:Himmelskoerper ;
        v:wirdUmrundetVon  :Deimos , :Phobos .

:Uranus  rdf:type  v:Himmelskoerper .

:Mercury  rdf:type  v:Himmelskoerper .

:Moon   rdf:type  v:Himmelskoerper .

:Enceladus  rdf:type  v:Himmelskoerper .

:Ganymede  rdf:type  v:Himmelskoerper .

:Saturn  rdf:type          v:Himmelskoerper ;
        v:wirdUmrundetVon  :Enceladus , :Mimas .

:Neptune  rdf:type  v:Himmelskoerper .

:Europa  rdf:type  v:Himmelskoerper .

:Callisto  rdf:type  v:Himmelskoerper .

:Earth  rdf:type           v:Himmelskoerper ;
        v:wirdUmrundetVon  :Moon .

:Sun    v:wirdUmrundetVon  :Saturn , :Uranus , :Earth , :Mars , :Mercury , :Neptune , :Venus , :Jupiter .

:Io     rdf:type  v:Himmelskoerper .
```


In [ ]:
constrGraph = sparql_construct(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

CONSTRUCT
{
   ?hkInstance rdf:type v:Himmelskoerper .
   ?orbitInstance v:wirdUmrundetVon ?sat
}
WHERE 
{
   {
      ?hkInstance rdf:type ?hk
      FILTER(?hk IN (dbo:Planet, dbo:Satellite))
      OPTIONAL {?sat rdf:type dbo:Satellite .
                ?sat v:orbits ?orbitInstance .}
   }
   UNION
   {
      ?sat rdf:type ?y .
      ?sat v:orbits :Sun .
      FILTER(?y IN (dbo:Planet, dbo:Satellite))
      VALUES(?orbitInstance){(:Sun)}
   }
   
}

""")
print (constrGraph.serialize(format='turtle'))

## Task 9 (1 pt)

Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist.
### Expected Result:
YES

In [ ]:
result = sparql_ask(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

ASK
{
   ?star rdf:type dbo:Star .
   ?star v:mass [rdf:value ?massVal] .
   FILTER(?massVal >1)
   FILTER(?star NOT IN (:Sun))  
}

""")
result

## Task 10 (1 pt)

Geben Sie eine Beschreibung der Resourcen aus, die einen Durchmesser zwischen 20.000 km und 30.000 km haben.

### Expected Result:
```
@prefix :     <http://dke.jku.at/example/solarsystem/> .
@prefix dbd:  <http://dbpedia.org/datatype/> .
@prefix dbo:  <http://dbpedia.org/ontology/> .
@prefix dbr:  <http://dbpedia.org/resource/> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sdo:  <http://schema.org/> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix v:    <http://dke.jku.at/example/examplevocabulary/> .
@prefix wd:   <http://www.wikidata.org/entity/> .
@prefix wdp:  <http://www.wikidata.org/prop/> .

:Uranus  rdf:type        dbo:Planet ;
        v:apoapsis       [ rdf:value  20.11 ;
                           v:uom      unit:AU
                         ] ;
        v:nrOfMoons      27 ;
        v:orbitalPeriod  [ rdf:value  84.0205 ;
                           v:uom      unit:YR
                         ] ;
        v:orbits         :Sun ;
        v:radius         [ rdf:value  25362 ;
                           v:uom      unit:KM
                         ] ;
        v:temperature    [ rdf:value  -197.2 ;
                           v:uom      unit:Deg_C
                         ] ;
        skos:exactMatch  dbr:Uranus .

:Neptune  rdf:type       dbo:Planet ;
        v:apoapsis       [ rdf:value  30.33 ;
                           v:uom      unit:AU
                         ] ;
        v:nrOfMoons      14 ;
        v:orbitalPeriod  [ rdf:value  164.8 ;
                           v:uom      unit:YR
                         ] ;
        v:orbits         :Sun ;
        v:radius         [ rdf:value  24622 ;
                           v:uom      unit:KM
                         ] ;
        v:temperature    [ rdf:value  -201 ;
                           v:uom      unit:Deg_C
                         ] ;
        skos:exactMatch  dbr:Neptune .
```


In [ ]:
descrGraph = sparql_construct(g,"""

base         <http://dke.jku.at/example/>
prefix :     <http://dke.jku.at/example/solarsystem/>
prefix v:    <http://dke.jku.at/example/examplevocabulary/>
prefix rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbr:  <http://dbpedia.org/resource/>
prefix dbo:  <http://dbpedia.org/ontology/>
prefix dbd:	 <http://dbpedia.org/datatype/>
prefix sdo:  <http://schema.org/>
prefix wd:   <http://www.wikidata.org/entity/>
prefix wdp:  <http://www.wikidata.org/prop/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix unit: <http://qudt.org/vocab/unit/>
prefix owl:  <http://www.w3.org/2002/07/owl#>

DESCRIBE ?r
WHERE 
{ 
   ?r v:radius [rdf:value ?rad]
   FILTER(?rad > 20000)
   FILTER(?rad < 30000)
}

""")
print (descrGraph.serialize(format='turtle'))